In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import tensorflow as tf

from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG

/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

/opt/anaconda3/lib/python3.12/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
/opt/anaconda3/lib/python3.12/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),


In [2]:
# Data Collection
ticker = 'AAPL'
stock_data = yf.download(ticker, start='2020-01-01', end='2024-01-01')
stock_data.info()

[*********************100%%**********************]  1 of 1 completed

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1006 entries, 2020-01-02 to 2023-12-29
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1006 non-null   float64
 1   High       1006 non-null   float64
 2   Low        1006 non-null   float64
 3   Close      1006 non-null   float64
 4   Adj Close  1006 non-null   float64
 5   Volume     1006 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 55.0 KB


In [3]:
# Setup Backtesting Environment
class SmaCross(Strategy):
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, 10)
        self.ma2 = self.I(SMA, price, 20)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()

# Apple data from yfinance
data = yf.download('AAPL', start='2015-01-01', end='2019-01-01')
bt = Backtest(data, SmaCross, commission=.002, exclusive_orders=True)
stats = bt.run()

# Plotting Results
bt.plot(filename='backtest_results.html', open_browser=True)

[*********************100%%**********************]  1 of 1 completed
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/opt/anaconda3/lib/python3.12/site-packages/b

GridPlot(id='p1325', ...)

In [4]:
import time
import numpy as np
import datetime as dt
import yfinance as yf

def calculate_rsi(prices, period=14):
    delta = np.diff(prices)
    gain = np.maximum(delta, 0)
    loss = np.abs(np.minimum(delta, 0))

    avg_gain = np.mean(gain[:period])
    avg_loss = np.mean(loss[:period])

    rsi = []
    for i in range(period, len(prices)):
        avg_gain = (avg_gain * (period - 1) + gain[i - period]) / period
        avg_loss = (avg_loss * (period - 1) + loss[i - period]) / period
        rs = avg_gain / avg_loss if avg_loss != 0 else 0
        rsi_value = 100 - (100 / (1 + rs))
        rsi.append(rsi_value)

    return rsi

def trade_yfinance():
    ticker = 'AAPL'
    data = yf.download(ticker, period='1mo', interval='1h')  # Download 1 month of hourly data
    close_prices = data['Close'].tolist()  # Convert the close prices to a list for simulation

    # Parameters for SMA calculation
    sma_short_period = 10
    sma_long_period = 20
    rsi_period = 14

    # Stop-loss and take-profit parameters (percentage)
    stop_loss_pct = 0.02  # 2% stop loss
    take_profit_pct = 0.04  # 4% take profit

    # Variables for simulation
    current_position = None
    cash = 100000  # Initial virtual cash
    stock_qty = 0
    entry_price = 0
    start_time = dt.datetime.now()
    max_duration = dt.timedelta(minutes=5)  # Simulate for 5 minutes
    index = 0
    total_profit = 0  # Track total profit or loss

    while True:
        # Check if the script has run for longer than the max duration
        if dt.datetime.now() - start_time > max_duration:
            print("Stopping the script after 5 minutes.")
            break

        print("Running simulation loop...")

        # Get the current price
        if index < len(close_prices):
            current_price = close_prices[index]
            index += 1
        else:
            print("End of historical price data.")
            break

        # Extract Close Prices for SMA Calculation (take the last `sma_long_period` prices)
        if index >= sma_long_period:
            recent_prices = close_prices[index - sma_long_period:index]

            # Calculate Moving Averages
            sma_short = np.mean(recent_prices[-sma_short_period:])
            sma_long = np.mean(recent_prices)

            # Calculate RSI
            if index >= sma_long_period + rsi_period:
                rsi_values = calculate_rsi(close_prices[index - (sma_long_period + rsi_period):index], rsi_period)
                rsi = rsi_values[-1]
            else:
                continue

            # Debug information for moving averages and RSI
            print(f"SMA Short: {sma_short}, SMA Long: {sma_long}, RSI: {rsi}")

            # Buy Signal with Relaxed RSI Confirmation
            if sma_short > sma_long and current_position is None and rsi < 40:
                qty = int(cash // current_price)
                if qty > 0:
                    stock_qty = qty
                    cash -= stock_qty * current_price
                    entry_price = current_price
                    current_position = 'long'
                    print(f"Bought {stock_qty} shares at {current_price}, Remaining Cash: {cash}")

            # Stop-loss or Take-profit check
            if current_position == 'long':
                # Stop-loss triggered
                if current_price <= entry_price * (1 - stop_loss_pct):
                    cash += stock_qty * current_price
                    profit = (current_price - entry_price) * stock_qty
                    total_profit += profit
                    print(f"Stop-Loss Triggered: Sold {stock_qty} shares at {current_price}, Profit: {profit}, New Cash Balance: {cash}")
                    stock_qty = 0
                    current_position = None

                # Take-profit triggered
                elif current_price >= entry_price * (1 + take_profit_pct):
                    cash += stock_qty * current_price
                    profit = (current_price - entry_price) * stock_qty
                    total_profit += profit
                    print(f"Take-Profit Triggered: Sold {stock_qty} shares at {current_price}, Profit: {profit}, New Cash Balance: {cash}")
                    stock_qty = 0
                    current_position = None

            # Sell Signal with Relaxed RSI Confirmation
            elif sma_short < sma_long and current_position == 'long' and rsi > 60:
                cash += stock_qty * current_price
                profit = (current_price - entry_price) * stock_qty
                total_profit += profit
                print(f"Sold {stock_qty} shares at {current_price}, Profit: {profit}, New Cash Balance: {cash}")
                stock_qty = 0
                current_position = None

            # Debug profit/loss tracking
            print(f"Total Profit/Loss so far: {total_profit}")

        time.sleep(1)

    # Final evaluation of performance
    if current_position == 'long':
        # Sell any remaining stock at the last available price
        cash += stock_qty * current_price
        profit = (current_price - entry_price) * stock_qty
        total_profit += profit
        print(f"Final Sale: Sold {stock_qty} shares at {current_price}, Profit: {profit}, New Cash Balance: {cash}")
        stock_qty = 0
        current_position = None

    print(f"Total Profit/Loss at the end of simulation: {total_profit}")

trade_yfinance()


[*********************100%%**********************]  1 of 1 completed


Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
SMA Short: 219.20724029541014, SMA Long: 217.69969482421874, RSI: 24.6019040024288

In [5]:
import time
import numpy as np
import datetime as dt
import yfinance as yf

def calculate_rsi(prices, period=14):
    delta = np.diff(prices)
    gain = np.maximum(delta, 0)
    loss = np.abs(np.minimum(delta, 0))

    avg_gain = np.mean(gain[:period])
    avg_loss = np.mean(loss[:period])

    rsi = []
    for i in range(period, len(prices)):
        avg_gain = (avg_gain * (period - 1) + gain[i - period]) / period
        avg_loss = (avg_loss * (period - 1) + loss[i - period]) / period
        rs = avg_gain / avg_loss if avg_loss != 0 else 0
        rsi_value = 100 - (100 / (1 + rs))
        rsi.append(rsi_value)

    return rsi

def trade_yfinance():
    ticker = 'AAPL'
    data = yf.download(ticker, period='1mo', interval='1h')  # Download 1 month of hourly data
    
    # Columns to use
    columns = ["High", "Low", "Open", "Close", "Volume", "Adj Close"]
    selected_data = data[columns].to_numpy()  # Convert selected columns to NumPy array

    # Extract individual columns
    high_prices = selected_data[:, 0]  # High prices
    low_prices = selected_data[:, 1]   # Low prices
    open_prices = selected_data[:, 2]  # Open prices
    close_prices = selected_data[:, 3] # Close prices
    volume = selected_data[:, 4]       # Volume
    adj_close_prices = selected_data[:, 5]  # Adjusted Close prices

    # Parameters for SMA calculation
    sma_short_period = 10
    sma_long_period = 20
    rsi_period = 14

    # Stop-loss and take-profit parameters (percentage)
    stop_loss_pct = 0.02  # 2% stop loss
    take_profit_pct = 0.04  # 4% take profit

    # Variables for simulation
    current_position = None
    cash = 100000  # Initial virtual cash
    stock_qty = 0
    entry_price = 0
    start_time = dt.datetime.now()
    max_duration = dt.timedelta(minutes=5)  # Simulate for 5 minutes
    index = 0
    total_profit = 0  # Track total profit or loss

    while True:
        # Check if the script has run for longer than the max duration
        if dt.datetime.now() - start_time > max_duration:
            print("Stopping the script after 5 minutes.")
            break

        print("Running simulation loop...")

        # Get the current price
        if index < len(close_prices):
            current_price = close_prices[index]
            index += 1
        else:
            print("End of historical price data.")
            break

        # Extract Close Prices for SMA Calculation (take the last `sma_long_period` prices)
        if index >= sma_long_period:
            recent_prices = close_prices[index - sma_long_period:index]

            # Calculate Moving Averages
            sma_short = np.mean(recent_prices[-sma_short_period:])
            sma_long = np.mean(recent_prices)

            # Calculate RSI
            if index >= sma_long_period + rsi_period:
                rsi_values = calculate_rsi(close_prices[index - (sma_long_period + rsi_period):index], rsi_period)
                rsi = rsi_values[-1]
            else:
                continue

            # Debug information for moving averages and RSI
            print(f"SMA Short: {sma_short}, SMA Long: {sma_long}, RSI: {rsi}")

            # Buy Signal with Relaxed RSI Confirmation
            if sma_short > sma_long and current_position is None and rsi < 40:
                qty = int(cash // current_price)
                if qty > 0:
                    stock_qty = qty
                    cash -= stock_qty * current_price
                    entry_price = current_price
                    current_position = 'long'
                    print(f"Bought {stock_qty} shares at {current_price}, Remaining Cash: {cash}")

            # Stop-loss or Take-profit check
            if current_position == 'long':
                # Stop-loss triggered
                if current_price <= entry_price * (1 - stop_loss_pct):
                    cash += stock_qty * current_price
                    profit = (current_price - entry_price) * stock_qty
                    total_profit += profit
                    print(f"Stop-Loss Triggered: Sold {stock_qty} shares at {current_price}, Profit: {profit}, New Cash Balance: {cash}")
                    stock_qty = 0
                    current_position = None

                # Take-profit triggered
                elif current_price >= entry_price * (1 + take_profit_pct):
                    cash += stock_qty * current_price
                    profit = (current_price - entry_price) * stock_qty
                    total_profit += profit
                    print(f"Take-Profit Triggered: Sold {stock_qty} shares at {current_price}, Profit: {profit}, New Cash Balance: {cash}")
                    stock_qty = 0
                    current_position = None

            # Sell Signal with Relaxed RSI Confirmation
            elif sma_short < sma_long and current_position == 'long' and rsi > 60:
                cash += stock_qty * current_price
                profit = (current_price - entry_price) * stock_qty
                total_profit += profit
                print(f"Sold {stock_qty} shares at {current_price}, Profit: {profit}, New Cash Balance: {cash}")
                stock_qty = 0
                current_position = None

            # Debug profit/loss tracking
            print(f"Total Profit/Loss so far: {total_profit}")

        time.sleep(1)

    # Final evaluation of performance
    if current_position == 'long':
        # Sell any remaining stock at the last available price
        cash += stock_qty * current_price
        profit = (current_price - entry_price) * stock_qty
        total_profit += profit
        print(f"Final Sale: Sold {stock_qty} shares at {current_price}, Profit: {profit}, New Cash Balance: {cash}")
        stock_qty = 0
        current_position = None

    print(f"Total Profit/Loss at the end of simulation: {total_profit}")

trade_yfinance()


[*********************100%%**********************]  1 of 1 completed


Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
SMA Short: 219.20724029541014, SMA Long: 217.69969482421874, RSI: 24.6019040024288

In [ ]:
import time
import numpy as np
import datetime as dt
import yfinance as yf

def calculate_rsi(prices, period=14):
    delta = np.diff(prices)
    gain = np.maximum(delta, 0)
    loss = np.abs(np.minimum(delta, 0))

    avg_gain = np.mean(gain[:period])
    avg_loss = np.mean(loss[:period])

    rsi = []
    for i in range(period, len(prices)):
        avg_gain = (avg_gain * (period - 1) + gain[i - period]) / period
        avg_loss = (avg_loss * (period - 1) + loss[i - period]) / period
        rs = avg_gain / avg_loss if avg_loss != 0 else 0
        rsi_value = 100 - (100 / (1 + rs))
        rsi.append(rsi_value)

    return rsi

def trade_finance():
    ticker = 'AAPL'
    data = yf.download(ticker, period='1mo', interval='1h')  # Download 1 month of hourly data

    # Columns to use
    columns = ["High", "Low", "Open", "Close", "Volume", "Adj Close"]
    selected_data = data[columns].to_numpy()  # Convert selected columns to NumPy array

    # Extract individual columns
    high_prices = selected_data[:, 0]  # High prices
    low_prices = selected_data[:, 1]  # Low prices
    open_prices = selected_data[:, 2]  # Open prices
    close_prices = selected_data[:, 3]  # Close prices

    # Parameters for SMA calculation
    sma_short_period = 10
    sma_long_period = 20
    rsi_period = 14

    # Stop-loss and take-profit parameters (percentage)
    stop_loss_pct = 0.02  # 2% stop loss
    take_profit_pct = 0.04  # 4% take profit

    # Variables for simulation
    current_position = None
    cash = 100000  # Initial virtual cash
    stock_qty = 0
    entry_price = 0
    start_time = dt.datetime.now()
    max_duration = dt.timedelta(minutes=5)  # Simulate for 5 minutes
    index = 0
    total_profit = 0  # Track total profit or loss

    while True:
        # Check if the script has run for longer than the max duration
        if dt.datetime.now() - start_time > max_duration:
            print("Stopping the script after 5 minutes.")
            break

        # Get the current price
        if index < len(close_prices):
            current_price = close_prices[index]
            high_price = high_prices[index]  # Use High price
            low_price = low_prices[index]    # Use Low price
            index += 1
        else:
            print("End of historical price data.")
            break

        # Extract Close Prices for SMA Calculation (take the last `sma_long_period` prices)
        if index >= sma_long_period:
            recent_prices = close_prices[index - sma_long_period:index]

            # Calculate Moving Averages
            sma_short = np.mean(recent_prices[-sma_short_period:])
            sma_long = np.mean(recent_prices)

            # Calculate RSI using Close prices
            if index >= sma_long_period + rsi_period:
                rsi_values = calculate_rsi(close_prices[index - (sma_long_period + rsi_period):index], rsi_period)
                rsi = rsi_values[-1]
            else:
                continue

            # Debug information for moving averages and RSI
            print(f"SMA Short: {sma_short}, SMA Long: {sma_long}, RSI: {rsi}")

            # Buy Signal with Relaxed RSI Confirmation
            if sma_short > sma_long and current_position is None and rsi < 40:
                qty = int(cash // current_price)
                if qty > 0:
                    stock_qty = qty
                    cash -= stock_qty * current_price
                    entry_price = current_price
                    current_position = 'long'
                    print(f"Bought {stock_qty} shares at {current_price}, Remaining Cash: {cash}")

            # Stop-loss or Take-profit check based on High and Low prices
            if current_position == 'long':
                # Stop-loss triggered based on Low price
                if low_price <= entry_price * (1 - stop_loss_pct):
                    cash += stock_qty * low_price
                    profit = (low_price - entry_price) * stock_qty
                    total_profit += profit
                    print(f"Stop-Loss Triggered: Sold {stock_qty} shares at {low_price}, Profit: {profit}, New Cash Balance: {cash}")
                    stock_qty = 0
                    current_position = None

                # Take-profit triggered based on High price
                elif high_price >= entry_price * (1 + take_profit_pct):
                    cash += stock_qty * high_price
                    profit = (high_price - entry_price) * stock_qty
                    total_profit += profit
                    print(f"Take-Profit Triggered: Sold {stock_qty} shares at {high_price}, Profit: {profit}, New Cash Balance: {cash}")
                    stock_qty = 0
                    current_position = None

            # Sell Signal with Relaxed RSI Confirmation
            elif sma_short < sma_long and current_position == 'long' and rsi > 60:
                cash += stock_qty * current_price
                profit = (current_price - entry_price) * stock_qty
                total_profit += profit
                print(f"Sold {stock_qty} shares at {current_price}, Profit: {profit}, New Cash Balance: {cash}")
                stock_qty = 0
                current_position = None

            # Debug profit/loss tracking
            print(f"Total Profit/Loss so far: {total_profit}")

        time.sleep(1)

    # Final evaluation of performance
    if current_position == 'long':
        # Sell any remaining stock at the last available price
        cash += stock_qty * current_price
        profit = (current_price - entry_price) * stock_qty
        total_profit += profit
        print(f"Final Sale: Sold {stock_qty} shares at {current_price}, Profit: {profit}, New Cash Balance: {cash}")
        stock_qty = 0
        current_position = None

    print(f"Total Profit/Loss at the end of simulation: {total_profit}")

trade_finance()


In [14]:
import time
import numpy as np
import datetime as dt
import yfinance as yf

def calculate_rsi(prices, period=14):
    delta = np.diff(prices)
    gain = np.maximum(delta, 0)
    loss = np.abs(np.minimum(delta, 0))

    avg_gain = np.mean(gain[:period])
    avg_loss = np.mean(loss[:period])

    rsi = []
    for i in range(period, len(prices)):
        avg_gain = (avg_gain * (period - 1) + gain[i - period]) / period
        avg_loss = (avg_loss * (period - 1) + loss[i - period]) / period
        rs = avg_gain / avg_loss if avg_loss != 0 else 0
        rsi_value = 100 - (100 / (1 + rs))
        rsi.append(rsi_value)

    return rsi

def trade_finance():
    ticker = 'AAPL'
    data = yf.download(ticker, period='1mo', interval='1h')  # Download 1 month of hourly data

    # Columns to use
    columns = ["High", "Low", "Open", "Close", "Volume", "Adj Close"]
    selected_data = data[columns].to_numpy()  # Convert selected columns to NumPy array

    # Extract individual columns
    high_prices = selected_data[:, 0]  # High prices
    low_prices = selected_data[:, 1]  # Low prices
    open_prices = selected_data[:, 2]  # Open prices
    close_prices = selected_data[:, 3]  # Close prices

    # Parameters for SMA calculation
    sma_short_period = 10
    sma_long_period = 20
    rsi_period = 14

    # Stop-loss and take-profit parameters (percentage)
    stop_loss_pct = 0.02  # 2% stop loss
    take_profit_pct = 0.04  # 4% take profit

    # Variables for simulation
    current_position = None
    cash = 100000  # Initial virtual cash
    stock_qty = 0
    entry_price = 0
    start_time = dt.datetime.now()
    max_duration = dt.timedelta(minutes=5)  # Simulate for 5 minutes
    index = 0
    total_profit = 0  # Track total profit or loss

    while True:
        # Check if the script has run for longer than the max duration
        if dt.datetime.now() - start_time > max_duration:
            print("Stopping the script after 5 minutes.")
            break

        # Get the current price
        if index < len(close_prices):
            current_price = close_prices[index]
            high_price = high_prices[index]  # Use High price
            low_price = low_prices[index]    # Use Low price
            index += 1
        else:
            print("End of historical price data.")
            break

        # Extract Close Prices for SMA Calculation (take the last `sma_long_period` prices)
        if index >= sma_long_period:
            recent_prices = close_prices[index - sma_long_period:index]

            # Calculate Moving Averages
            sma_short = np.mean(recent_prices[-sma_short_period:])
            sma_long = np.mean(recent_prices)

            # Calculate RSI using Close prices
            if index >= sma_long_period + rsi_period:
                rsi_values = calculate_rsi(close_prices[index - (sma_long_period + rsi_period):index], rsi_period)
                rsi = rsi_values[-1]
            else:
                continue

            # Debug information for moving averages and RSI
            print(f"SMA Short: {sma_short}, SMA Long: {sma_long}, RSI: {rsi}")

            # Buy Signal with Relaxed RSI Confirmation
            if sma_short > sma_long and current_position is None and rsi < 40:
                qty = int(cash // current_price)
                if qty > 0:
                    stock_qty = qty
                    cash -= stock_qty * current_price
                    entry_price = current_price
                    current_position = 'long'
                    print(f"Bought {stock_qty} shares at {current_price}, Remaining Cash: {cash}")

            # Stop-loss or Take-profit check based on High and Low prices
            if current_position == 'long':
                # Stop-loss triggered based on Low price
                if low_price <= entry_price * (1 - stop_loss_pct):
                    cash += stock_qty * low_price
                    profit = (low_price - entry_price) * stock_qty
                    total_profit += profit
                    print(f"Stop-Loss Triggered: Sold {stock_qty} shares at {low_price}, Profit: {profit}, New Cash Balance: {cash}")
                    stock_qty = 0
                    current_position = None

                # Take-profit triggered based on High price
                elif high_price >= entry_price * (1 + take_profit_pct):
                    cash += stock_qty * high_price
                    profit = (high_price - entry_price) * stock_qty
                    total_profit += profit
                    print(f"Take-Profit Triggered: Sold {stock_qty} shares at {high_price}, Profit: {profit}, New Cash Balance: {cash}")
                    stock_qty = 0
                    current_position = None

            # Sell Signal with Relaxed RSI Confirmation
            elif sma_short < sma_long and current_position == 'long' and rsi > 60:
                cash += stock_qty * current_price
                profit = (current_price - entry_price) * stock_qty
                total_profit += profit
                print(f"Sold {stock_qty} shares at {current_price}, Profit: {profit}, New Cash Balance: {cash}")
                stock_qty = 0
                current_position = None

            # Debug profit/loss tracking
            print(f"Total Profit/Loss so far: {total_profit}")

        time.sleep(1)

    # Final evaluation of performance
    if current_position == 'long':
        # Sell any remaining stock at the last available price
        cash += stock_qty * current_price
        profit = (current_price - entry_price) * stock_qty
        total_profit += profit
        print(f"Final Sale: Sold {stock_qty} shares at {current_price}, Profit: {profit}, New Cash Balance: {cash}")
        stock_qty = 0
        current_position = None

    print(f"Total Profit/Loss at the end of simulation: {total_profit}")

trade_finance()


[*********************100%%**********************]  1 of 1 completed


SMA Short: 219.20724029541014, SMA Long: 217.69969482421874, RSI: 24.60190400242884
Bought 451 shares at 221.5, Remaining Cash: 103.5
Total Profit/Loss so far: 0
SMA Short: 219.65924072265625, SMA Long: 217.90119476318358, RSI: 25.673006128694183
Total Profit/Loss so far: 0
SMA Short: 220.99374084472657, SMA Long: 218.54811935424806, RSI: 24.29025852625412
Total Profit/Loss so far: 0
SMA Short: 222.1817413330078, SMA Long: 219.20261993408204, RSI: 21.62208159553124
Total Profit/Loss so far: 0
SMA Short: 223.396240234375, SMA Long: 219.85411987304687, RSI: 23.993302034937884
Total Profit/Loss so far: 0
SMA Short: 224.23975067138673, SMA Long: 220.49112548828126, RSI: 24.0005827626252
Total Profit/Loss so far: 0
SMA Short: 225.06374969482422, SMA Long: 221.12337493896484, RSI: 28.17200874557571
Total Profit/Loss so far: 0
SMA Short: 225.9294692993164, SMA Long: 221.78173446655273, RSI: 31.093202411830774
Total Profit/Loss so far: 0
SMA Short: 226.69046020507812, SMA Long: 222.40173492431